In [43]:
import pandas as pd
import numpy as np
from persiantools import characters, digits
import jdatetime
import re
from glob import glob
import os

In [44]:
nemad = pd.read_excel(r'D:\Python_project\FirmFSCleansing\raw\FilesForCleaningFunction\Nemad_info.xlsx')

In [45]:
nemad.fa = nemad.fa.apply(lambda x : characters.ar_to_fa(x))

In [46]:
nemad.rename(columns={'stock_name':'stock_name_plus','fa':'stock_name'}, inplace=True)

In [47]:
def file_path(path):
    if __name__ == "__main__":
        balance_sheet = []
        profit_loss = []
        for (root, dirs, files) in os.walk(path , topdown = True):
            if len(files) == 0:
                continue
            else:
                balance_sheet.append(root + '\\' + files[0])
                profit_loss.append(root + '\\' + files[1])
        return balance_sheet, profit_loss

In [48]:
balance_sheet , profit_loss = file_path(r'D:\Python_project\FirmFSCleansing\raw\CodalRawUpdatingDate_14011026')

# check_shape_befor_run

In [49]:
# def shape(kk):
#     l=pd.read_excel(kk)
#     a = l.shape
    
#     return a

In [50]:
# pd.Series(balance_sheet).apply(shape).unique()

In [51]:
# abc = pd.Series(balance_sheet).apply(shape)==(13, 11)
# abc.to_excel(r'C:\Users\cmos\Desktop\abc.xlsx')

In [52]:
# balance_sheet[3701]

# type_1_function

In [53]:
def clean_drop_1(w):
    w.drop(w.columns[0], axis=1, inplace= True)
    w.drop(w.columns[4], axis=1, inplace= True)
    w.drop(w.columns[4], axis=1, inplace= True)
    w.dropna(axis=1 , how="all" , inplace=True)

    w.drop([1,3] , axis=0 , inplace=True)
    
    data_report = re.search(r'(\d+/\d+/\d+)', w.loc[0 , 'title']).group(1)
    w.rename(columns={0:"description" , 
#                     1:f"reported_period {str(digits.fa_to_en(w.iloc[0,1].split(' ')[-1]))}" ,
                    1:f"reported_period {str(digits.fa_to_en(data_report))}" ,
                    2:f"variable_period {str(digits.fa_to_en(w.iloc[0,2].split(' ')[-1]))}" ,
                    3:f"variable_period {str(digits.fa_to_en(w.iloc[0,3].split(' ')[-1]))}" 
                    } , inplace=True)

    return w

In [54]:
def new_col_1(i):
    i.insert(loc=2 , column='date_1' , value= re.sub("\/" , "-" , digits.fa_to_en(list(i.columns)[1].split(' ')[-1])))
    i.insert(loc=4 , column='date_2' , value= re.sub("\/" , "-" , digits.fa_to_en(list(i.columns)[3].split(' ')[-1])))
    i.insert(loc=6 , column='date_3' , value= re.sub("\/" , "-" , digits.fa_to_en(list(i.columns)[5].split(' ')[-1])))

    i.drop([0] , axis=0 , inplace=True)
    i.reset_index(drop=True , inplace=True)

    # i.drop(columns=['percent_change'] , inplace=True)
    
    
    
    nan_index= i[i['description'].isna()].index
    nan_index= nan_index.to_list()
    i.loc[nan_index , 'description']=['total_current_assets_befor' , 'total_current_liabilities_befor']

    
    
    i = i.replace(np.nan , '0')

    if 'حسابرسی شده' in i.iloc[0,3]:
        i.insert(loc=7 , column='audit_status_3' , value= 'audited')
    elif 'حسابرسی شده' in i.iloc[0,1]:
        i.insert(loc=7 , column='audit_status_3' , value= 'audited')
    else:
        i.insert(loc=7 , column='audit_status_3' , value= 'not_audited')
    if 'حسابرسی شده' in i.iloc[0,1]:
        i.insert(loc=5 , column='audit_status_2' , value= 'audited')
    else:
        i.insert(loc=5 , column='audit_status_2' , value= 'not_audited')
    if 'حسابرسی شده' in i.iloc[0,11]:
        i.insert(loc=3 , column='audit_status_1' , value= 'audited')
    else:
        i.insert(loc=3 , column='audit_status_1' , value= 'not_audited')

    i.drop([0] , axis=0 , inplace=True)
    i.reset_index(drop=True , inplace=True)

    return i

In [55]:
def label_trans_1(y):

    y['description'].replace({'دارایی ها':'assets',
    'دارایی‌‌ها':'assets',
    'دارایی‌ها':'assets',
    'دریافتنی‌‌های بلندمدت':'long_term_receivables',
    'سرمایه‌گذاری‌‌های کوتاه‌مدت':'short_run_investment',
    'دارایی‌‌های جاری':'current_assets',
    'دارايی‌های غیرجاری':'non_current_assets',
    'دارایی‌های ثابت مشهود':'tangible_fixed_assets',
    'سرمایه‌گذاری در املاک':'real_estate_investments',
    'دارایی‌های نامشهود':'intangible_assets',
    'سرمایه‌گذاری‌های بلندمدت':'long_run_investments',
    'دریافتنی‌های بلندمدت':'long_term_receivables',
    'سایر دارایی‌ها':'other_assets',
    'جمع دارایی‌های غیرجاری':'total_non_current_assets',
    'دارایی‌های جاری':'current_assets',
    'سفارشات و پیش‌پرداخت‌ها':'orders_and_prepayments',
    'موجودی مواد و کالا':'inventories',
    'دریافتنی‌های تجاری و سایر دریافتنی‌ها':'commercial_and_other_receivables',
    'سرمایه‌گذاری‌های کوتاه‌مدت':'short_run_investment',
    'موجودی نقد':'cash',
    'جمع دارایی های جاری':'total_current_assets',
    'جمع دارایی‌های جاری':'total_current_assets',
    'دارایی‌های نگهداری شده برای فروش':'Assets_held_for_sale',
    'جمع دارایی‌ها':'total_assets',
    'حقوق مالکانه و بدهی‌ها':'liabilities_and_equity',
    'حقوق مالکانه':'equity',
    'سرمايه':'capital',
    'افزایش سرمایه در جریان':'capital_increase_in_process',
    'صرف سهام':'sarf_saham',
    'صرف سهام خزانه':'sarf_saham_khazane',
    'اندوخته قانونی':'legal_reserved',
    'ساير اندوخته‌ها':'other_reserves',
    'مازاد تجدیدارزيابی دارایی‌ها':'assets_revaluation_surplus',
    'تفاوت تسعیر ارز عملیات خارجی':'tafavot_taseer_arz',
    'سود (زيان) انباشته':'accumulated_(loss)_profit',
    'سود انباشته':'accumulated_(loss)_profit',
    'سهام خزانه':'saham_khazane',
    'جمع حقوق مالکانه':'total_equity',
    'بدهی‌ها':'liabilities',
    'بدهی‌های غیرجاری':'non_current_liabilities',
    'پرداختنی‌های بلندمدت':'long_term_payables',
    'تسهیلات مالی بلندمدت':'long_term_loans',
    'ذخیره مزایای پایان خدمت کارکنان':'pention_reserves',
    'جمع بدهی‌های غیرجاری':'total_non_current_liabilities',
    'بدهی‌های جاری':'current_liabilities',
    'پرداختنی‌های تجاری و سایر پرداختنی‌ها':'commercial_payables_and_other_payables',
    'مالیات پرداختنی':'payable_taxes',
    'سود سهام پرداختنی':'payable_dividends',
    'تسهیلات مالی':'short_term_loans',
    'ذخایر':'reserves',
    'پیش‌دریافت‌ها':'pish_daryaftha',
    'بدهی‌های ‌مرتبط ‌با دارایی‌های نگهداری‌‌شده برای ‌فروش':'liabilities_related_assets_held_for_sale',
    'جمع بدهی‌های جاری':'total_current_liabilities',
    'جمع بدهی‌ها':'total_liabilities',
    'جمع حقوق مالکانه و بدهی‌ها':'total_liabilities_and_equity',
    'موجودي نقد':'cash',
    'مطالبات از بانک ها و ساير موسسات اعتباري':'interbank_claim',
    'مطالبات از دولت':'gov_claim',
    'تسهيلات اعطايي و مطالبات از اشخاص دولتي':'public_claim',
    'تسهيلات اعطايي و مطالبات از اشخاص غير دولتي':'private_claim',
    'سرمایه‌گذاری در سهام و ساير اوراق بهادار':'securities_investment',
    'مطالبات از شرکت‌های فرعی و وابسته':'subsidiaries_claim',
    'ساير حسابهاي دريافتني':'other_revceivables',
    'دارایی‌هاي ثابت مشهود':'tangible_fixed_assets',
    'دارایی‌هاي نا مشهود':'intangible_assets',
    'سپرده قانونی':'legal_deposit',
    'ساير دارایی‌ها':'other_assets',
    'افزايش سرمايه در جريان':'capital_increase_in_process',
    'اندوخته صرف سهام':'andokhte_sarf_saham',
    'اندوخته قانوني':'legal_reserved',
    'مازاد تجديد ارزيابي دارايي‌ها':'assets_revaluation_surplus',
    'تفاوت تسعیر ارز':'tafavot_taseer_arz',
    'بدهي به بانكها و ساير موسسات اعتباري':'interbank_liabilities',
    'سپرده‌هاي مشتریان':'customer_deposits',
    'سود سهام پرداختني':'payable_dividends',
    'اوراق بدهی':'bond',
    'ذخيره ماليات عملکرد':'operation_tax_reserve',
    'ذخاير و ساير بدهی‌ها':'reserves_and_other_liabilities',
    'ذخيره مزاياي پايان خدمت و تعهدات بازنشستگی کارکنان':'pention_reserves',
    'جمع بدهی های قبل از حقوق صاحبان سپرده های سرمایه گذاری':'total_liabilities_befor_investment_deposits_owners_rights',
    'حقوق صاحبان سپرده‌های سرمایه‌گذاری':'investment_deposits_owners_rights',
    'جمع بدهی‌ها و حقوق مالکانه':'total_liabilities_and_equity',
    'دارایی های ثابت مشهود':'tangible_fixed_assets',
    'دارایی ها نامشهود':'intangible_assets',
    'مطالبات از شرکت های فرعی و وابسته':'subsidiaries_claim',
    'سرمایه گذاری در سهام و سایر اوراق بهادار':'securities_investment',
    'سایر حساب های دریافتنی':'other_receivables',
    'سایر دارایی ها':'other_assets',
    'مطالبات دولت':'gov_claim',
    'تسهیلات اعطایی و مطالبات از اشخاص غیر دولتی':'private_claim',
    'تسهیلات اعطایی و مطالبات از اشخاص دولتی':'public_claim',
    'مطالبات از بانک ها و سایر موسسات اعتباری':'interbank_claim',
    'ذخایر و سایر بدهی ها':'reserves_and_other_liabilities',
    'ذخیره مالیات عملکرد':'tax_reserve',
    'سپرده های مشتریان':'customer_deposits',
    'بدهی به بانک ها و سایر موسسات اعتباری':'interbank_liabilities',
    'سرمایه‌گذاری‌‌های کوتاه مدت':'short_run_investment',
    'دریافتنی‌‌های تجاری':'commercial_receivables',
    'دریافتنی‌‌های غیرتجاری':'non_commercial_receivables',
    'پیش پرداخت‌ها و سفارشات':'orders_and_prepayments',
    'دارایی‌‌های غیرجاری':'non_current_assets',
    'زمین نگهداری شده برای ساخت املاک':'land_held_for_construction',
    'بدهی‌ها و حقوق صاحبان سهام':'liabilities_and_equity',
    'پرداختنی‌های تجاری':'commercial_payables',
    'پرداختنی‌های غیرتجاری':'non_commercial_payables',
    'پیش‌دریافت‌های جاری':'current_pish_daryaftha',
    'بدهی‌های مرتبط با دارایی‌های نگهداری شده برای فروش':'liabilities_related_assets_held_for_sale',
    'پیش‌دریافت‌های غیرجاری':'non_current_pish_daryaftha',
    'حقوق صاحبان سهام':'equity',
    'سرمایه':'capital',
    'افزایش (کاهش) سرمایه در جریان':'capital_increase_in_process',
    'صرف (کسر) سهام':'sarf_saham',
    'سایر اندوخته‌ها':'other_reserves',
    'مازاد تجدید ارزیابی دارایی‌های نگهداری شده برای فروش':'revaluation_surplus_assets_held_for_sale',
    'مازاد تجدید ارزیابی دارایی‌ها':'assets_revaluation_surplus',
    'تفاوت تسعیر ناشی از تبدیل به واحد پول گزارشگری':'tafavot_taseer_arz',
    'اندوخته تسعیر ارز دارایی‌ها و بدهی‌های شرکت‌های دولتی':'andokhte_taseer_arz-sherkat_dolati',
    'سود (زیان) انباشته':'accumulated_(loss)_profit',
    'جمع حقوق صاحبان سهام':'total_equity',
    'جمع بدهی‌ها و حقوق صاحبان سهام':'total_liabilities_and_equity',
    'مازاد تجدید ارزیابی دارایی‌های غیرجاری نگهداری شده برای فروش':'revaluation_surplus_non_current_assets_held_for_sale',
    'اندوخته تسعير ارز دارايي‌ها و بدهي‌هاي شرکت‌هاي دولتي':'andokhte_taseer_arz-sherkat_dolati',
    'سرمایه گذاریهای کوتاه مدت':'short_run_investment',
    'حسابها و اسناد دریافتنی تجاری':'commercial_and_other_receivables',
    'طلب از شرکتهای گروه و شرکتهای وابسته':'subsidiaries_claim',
    'سایر حسابها و اسناد دریافتنی':'other_receivables',
    'سفارشات':'orders',
    'پیش پرداختها':'prepayment',
    'دارایی های غیر جاری نگهداری شده برای فروش':'non_current_assets_held_for_sale',
    'دارایی های غیرجاری':'non_current_assets',
    'حسابها و اسناد دریافتنی بلند مدت':'long_term_receivables',
    'سرمایه گذاریهای بلند مدت':'long_run_investments',
    'جمع داراییهای غیر جاری':'total_non_current_assets',
    'حصه جاری حسابها و اسناد دریافتنی تجاری':'short_term_receivables',
    'داراییهای ثابت پس از کسر استهلاک':'fixes_assets_after_Depreciation',
    'حسابها و اسناد دریافتنی تجاری(بلند مدت)':'long_term_receivables',
    'پیش پرداختهای سرمایه ای':'capital_prepayments',
    'مطالبات از بانک‌های مرکزی':'cbi_claims',
    'مطالبات از بانک‌ها و سایر موسسات اعتباری':'interbank_claim',
    'تسهیلات اعطایی و مطالبات از اشخاص دولتی به غیر از بانک‌ها':'public_claim',
    'تسهیلات اعطایی و مطالبات از اشخاص غیردولتی به غیر از بانک‌ها':'private_claim',
    'سرمایه‌گذاری در سهام و سایر اوراق بهادار':'securities_investment',
    'مطالبات از بانک مرکزی':'cbi_claims',
    'مطالبات از سایر بانکها و موسسات اعتباری':'interbank_claim',
    'تسهیلات اعطایی به شرکتهای عضو گروه':'subsidiaries_claim',
    'تسهیلات اعطایی به سایر اشخاص':'private_claim',
    'سایر حسابهای دریافتنی':'other_receivables',
    'دارایی‌های غیر جاری نگهداری شده برای فروش':'non_current_assets_held_for_sale',
    'اوراق مشارکت':'corporate_bond',
    'سرمایه گذاریها':'investments',
    'حسابها و اسناد دریافتنی تجاری (بلند مدت)':'long_term_receivables',
    'بدهی های جاری':'current_liabilities',
    'حسابها و اسناد پرداختنی تجاری':'commercial_payables',
    'بدهی به شرکتهای گروه و وابسته':'subsidiaries_liabilities',
    'سایر حسابها و اسناد پرداختنی':'other_payables',
    'پیش دریافتها':'pish_daryaftha',
    'ذخیره مالیات':'tax_reserve',
    'تسهیلات مالی دریافتی':'short_term_loans',
    'بدهی های مرتبط با دارایی های غیر جاری نگهداری شده برای فروش':'liabilities_related_non_current_assets_held_for_sale',
    'بدهی های غیر جاری':'non_current_liabilities',
    'حسابها و اسناد پرداختنی بلند مدت':'long_term_payables',
    'تسهیلات مالی دریافتی بلند مدت':'long_term_loans',
    'جمع بدهیهای غیر جاری':'total_non_current_liabilities',
    'جمع بدهی ها':'total_liabilities',
    'دریافتی بابت افزایش سرمایه':'funds_received_for_capital_increase',
    'سایر اندوخته ها':'other_reserves',
    'ذخیره مالیات بر درآمد':'tax_reserve',
    'حصه جاری تسهیلات مالی دریافتی':'short_term_loans',
    'سود سهام پیشنهادی و پرداختنی':'payable_dividends',
    'ذخیره مزایای پایان خدمت':'pention_reserves',
    'جمع بدهیهای جاری و غیر جاری':'total_liabilities',
    'وجوه دریافتی بابت افزایش سرمایه':'funds_received_for_capital_increase',
    'اندوخته طرح و توسعه':'r&d_reserve',
    'بدهی به بانک مرکزی و صندوق توسعه ملی':'ndf_cbi_liabilities',
    'بدهی به بانک‌ها و سایر موسسات اعتباری':'interbank_liabilities',
    'سپرده‌های دیداری و مشابه':'current_deposits',
    'سپرده‌های پس‌انداز و مشابه':'savings_deposits',
    'سپرده‌های سرمایه‌گذاری مدت‌دار':'investment_deposits',
    'سایر سپرده‌ها':'other_deposits',
    'سایر بدهی‌ها':'other_liabilities',
    'بدهی به بانک مرکزی':'cbi_liabilities',
    'بدهی به بانکها و موسسات اعتباری':'interbank_liabilities',
    'سپرده های دیداری':'current_deposits',
    'سپرده های قرض الحسنه و پس اندار و مشابه':'savings_deposits',
    'سپرده های سرمایه گذاری مدت دار':'investment_deposits',
    'سایر سپرده ها':'other_deposits',
    'سود پرداختنی به سپرده گذاران':'depositors_dividend',
    'بدهی‌های مرتبط با دارایی‌های غیر جاری نگهداری شده برای فروش':'liabilities_related_non_current_assets_held_for_sale',
    'دخیره مزایای پایان خدمت کارکنان':'pention_reserves',
    'مطالبات از بیمه‌گذاران و نمایندگان':'claims_on_insurers_and_agents',
    'مطالبات از بیمه‌گران و بیمه‌گران اتکایی':'Claims_on_ insurers_and_reinsurers',
    'سهم بیمه‌گران اتکایی از ذخایر فنی':'Insurers_share_of_relying_on_technical_reserves',
    'سایر دریافتنی‌ها و پیش‌پرداخت‌ها':'other_receivables',
    'تسهیلات اعطایی به اشخاص':'private_loans',
    'سرمایه‌گذاری‌ در اوراق بهادار وسپرده های بانکی':'securities_and_bank_deposits_invstments',
    'سرمایه گذاری در املاک':'real_estate_investments',
    'سایر سرمایه گذاری ها':'other_investments',
    'دارایی های غیرجاری نگهداری شده برای فروش':'non_current_assets_held_for_sale',
    'دارایی‌های نا مشهود':'intangible_assets',
    'مطالبات از بيمه‌گذاران و نمايندگان':'claims_on_insurers_and_agents',
    'مطالبات از بیمه گران و بيمه‌گران اتکايی':'Claims_on_ insurers_and_reinsurers',
    'سهم بیمه گران اتكایی از ذخایر فنی':'Insurers_share_of_relying_on_technical_reserves',
    'مطالبات بلندمدت':'long_term_claims',
    'بدهي به بيمه‌گذاران و نمايندگان':'liabilities_on_insurers_and_agents',
    'ذخاير حق بيمه':'insurance_premium_reserves',
    'ذخيره خسارت معوق':'deferred_damages_reserves',
    'ذخیره ریسک‌های منقضی نشده':'not_expired_reisl_reserves',
    'حق بيمه سال‌های آتی':'future_years_insurance_premium',
    'جمع دارايي‌ها':'total_assets',
    'اندوخته سرمایه ای':'capital_reserves',
    'بدهي‌ها و حقوق مالکانه':'liabilities_and_equity',
    'بدهی به بیمه‌گذاران و نمایندگان':'liabilities_on_insurers_and_agents',
    'بدهی به بیمه‌گران و بیمه‌گران اتکایی':'liabilities_on_insurers_and_reinsurers',
    'سایر پرداختنی‌ها':'other_payables',
    'بدهی بابت بیمه عمر و مدیریت سرمایه':'life_insurance_and_capital_management_liabilities',
    'بدهی مرتبط با دارایی های غیرجاری نگهداری شده برای فروش':'liabilities_related_non_current_assets_held_for_sale',
    'ذخیره خسارت معوق':'deferred_damages_reserves',
    'ذخیره ریسکهای منقضی نشده':'not_expired_reisl_reserves',
    'ذخیره حق بیمه عاید نشده':'unearned_premium_reserves',
    'ذخیره ریاضی بیمه­ عمر و تشکیل سرمایه':'mathematical_reserves_of_life_insurance_and_capital_formation',
    'سایر ذخایر فنی':'other_technical_reserves',
    'ذخيره مزاياي پايان خدمت کارکنان':'pention_reserves',
    'اندوخته سرمایه‌ای':'capital_reserves',
    'تأثير تغييرات نرخ ارز':'exchange_rate_changes_effect',
    'جمع بدهي‌ها و حقوق مالکانه':'total_liabilities_and_equity',
    'مازاد تجديدارزيابي دارايي‌ها':'assets_revaluation_surplus',
    'جمع داراییهای جاری':'total_current_assets',
    'داراییهای نامشهود':'intangible_assets',
    'سایر داراییها':'other_assets',
    'جمع داراییها':'total_assets',
    'جمع بدهیهای جاری':'total_current_liabilities',
    'جمع بدهیها و حقوق صاحبان سهام':'total_liabilities_and_equity',
    'داراییهای ثابت مشهود':'tangible_fixed_assets',
    'مازاد تجدید ارزیابی داراییها':'assets_revaluation_surplus',
    'سایر حساب‌ها و اسناد دریافتنی':'other_receivables',
    'سرمایه گذاری‌های کوتاه مدت':'short_run_investment',
    'طلب از شرکت‌های گروه و شرکت‌های وابسته':'subsidiaries_claim',
    'حصه جاری حساب‌ها و اسناد دریافتنی تجاری':'short_term_receivables',
    'پیش پرداخت‌ها':'prepayment',
    'سرمایه گذاری‌های بلند مدت':'long_run_investments',
    'دارایی‌های ثابت پس از کسر استهلاک':'fixes_assets_after_Depreciation',
    'پیش پرداخت‌های سرمایه ای':'capital_prepayments',
    'جمع دارایی‌های غیر جاری':'total_non_current_assets',
    'حساب‌ها و اسناد پرداختنی تجاری':'commercial_payables',
    'سایر حساب‌ها و اسناد پرداختنی':'other_payables',
    'بدهی به شرکت‌های گروه و وابسته':'subsidiaries_liabilities',
    'جمع بدهی‌های غیر جاری':'total_non_current_liabilities',
    'جمع بدهی‌های جاری و غیر جاری':'total_liabilities',
    'اندوخته تسعیر ارز داراییها و بدهیهای شرکتهای دولتی':'andokhte_taseer_arz-sherkat_dolati',
    'سرمایه گذاری‌های کوتاه مدت':'short_run_investment',
    'طلب از شرکت‌های گروه و شرکت‌های وابسته':'subsidiaries_claim',
    'حصه جاری حساب‌ها و اسناد دریافتنی تجاری':'short_term_receivables',
    'سایر حساب‌ها و اسناد دریافتنی':'other_receivables',
    'پیش پرداخت‌ها':'prepayment',
    'سرمایه گذاری‌های بلند مدت':'long_run_investments',
    'دارایی‌های ثابت پس از کسر استهلاک':'fixes_assets_after_Depreciation',
    'پیش پرداخت‌های سرمایه ای':'capital_prepayments',
    'سایر حساب‌ها و اسناد پرداختنی':'other_payables',
    'بدهی به شرکت‌های گروه و وابسته':'subsidiaries_liabilities',
    'جمع بدهی‌های غیر جاری':'total_non_current_liabilities',
    'جمع بدهی‌های جاری و غیر جاری':'total_liabilities',
    'حسابها و اسناد پرداختنی تجاری':'commercial_payables',
    'حسابها و اسناد پرداختنی تجاری':'commercial_payables',
    } , inplace=True)



    
    
#     y = y.replace(np.nan , '0')
    
    return y

In [56]:
def col_edit_1(t):

    
    t.insert(loc= 10, column= "national_code", value= list(nemad.loc[nemad['stock_name']== t.loc[0 , 'nemad']]['shenaseh_meli'])[0])
    t.replace(t['company_name'].values, list(nemad.loc[nemad['stock_name']== t.loc[0 , 'nemad'] ]['company_name_English'])[0], inplace=True)
    t.replace(t['nemad'].values, list(nemad.loc[nemad['stock_name']== t.loc[0 , 'nemad']]['en'])[0] , inplace=True)
    
#     t.insert(loc= 10, column= "national_code", value= list(nemad.loc[nemad['stock_name']== t.loc[0 , 'nemad']]['shenaseh_meli'])[0])
#     t.replace(t['nemad'].values, list(nemad.loc[nemad['stock_name']== t.loc[0 , 'nemad']]['en'])[0] , inplace=True)
#     t.replace(t['company_name'].values, list(nemad.loc[nemad['stock_name']== 'آ س پ' ]['company_name_English'])[0], inplace=True)
    

    t.replace(t['table_title'].values, "balance_sheet" , inplace=True)

    if t.loc[0,'title'].find('دوره ۶ ماهه') != -1:
        t.insert(loc=13 , column = 'reported_period' , value = "6_months_interim")
    elif t.loc[0,'title'].find('دوره ۳ ماهه') != -1:
        t.insert(loc=13 , column='reported_period' , value= "3_months_interim")
    elif t.loc[0,'title'].find('دوره ۹ ماهه') != -1:
        t.insert(loc=13 , column='reported_period' , value= "9_months_interim")
    else:
        t.insert(loc=13 , column='reported_period' , value= "12_months_interim")

    return t

In [57]:
def to_g(x):
    x=jdatetime.datetime(int(x.split('-')[0]) , int(x.split('-')[1]),int(x.split('-')[2])).togregorian()
    return x

In [58]:
def time_edit_1(r):
    r.insert(loc=14 , column="reported_date_j" , value =re.sub("\/" , "-" , digits.fa_to_en(re.search(r'(\d+/\d+/\d+)', r.loc[0 , 'title']).group(1))) )
    r.insert(loc=15 , column="reported_date_m" , value= r['reported_date_j'].apply(lambda x:to_g(x)))
    return r

In [59]:
def char_edit_1(ch):

    if ch.loc[0,'title'].find('حسابرسی شده') != -1:
        ch.insert(loc=16 , column='reported_audit_status' , value= "audited")
    else:
        ch.insert(loc=16 , column='reported_audit_status' , value= "not_audited")

    if ch.loc[0 , 'title'].find('اصلاحیه') != -1:
        ch.insert(loc=17 , column= 'is_corrected' , value='1')
    else:
        ch.insert(loc=17 , column= 'is_corrected' , value='0')
        
    
    ch.drop(columns=['title'] , inplace=True)

    # ch= ch.replace(np.nan , '0')
    ch[ch.columns[4]]= ch[ch.columns[4]].apply(lambda x: digits.fa_to_en(x))
    ch[ch.columns[7]]= ch[ch.columns[7]].apply(lambda x: digits.fa_to_en(x))
    ch[ch.columns[1]]= ch[ch.columns[1]].apply(lambda x: digits.fa_to_en(x))

    # ch['reported_date_j']= ch['reported_date_j'].apply(lambda x: re.sub("\/" , "-" , digits.fa_to_en(x))     #digits.fa_to_en(ch.iloc[0,5].split(' ')[-1]))  re.sub("\/" , "-" , digits.fa_to_en(data.iloc[0,14].split(' ')[-3])))
    ch['publish_time']= ch['publish_time'].apply(lambda x: digits.fa_to_en(x))
    ch['send_time']= ch['send_time'].apply(lambda x: digits.fa_to_en(x))



    return ch

In [60]:
def x_concat_1(table):

    table_1= table.filter(['description' , table.columns[1] , 'date_1' , 'audit_status_1' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'reported_audit_status' , 'is_corrected' , 'talfighi' , 'send_time' , 'publish_time' , 'table_title'])
    table_2= table.filter(['description' , table.columns[4] , 'date_2' , 'audit_status_2' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'reported_audit_status' , 'is_corrected' , 'talfighi' ,'send_time' , 'publish_time' , 'table_title'])
    table_3= table.filter(['description' , table.columns[7] , 'date_3' , 'audit_status_3' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'reported_audit_status' , 'is_corrected' , 'talfighi' ,'send_time' , 'publish_time' , 'table_title'])

    table_1.rename(columns={table.columns[1]:'value' , 'date_1':'date' , 'audit_status_1':'audit_status'} , inplace=True)
    table_2.rename(columns={table.columns[4]:'value' , 'date_2':'date' , 'audit_status_2':'audit_status'} , inplace=True)
    table_3.rename(columns={table.columns[7]:'value' , 'date_3':'date' , 'audit_status_3':'audit_status'} , inplace=True)

    data_final= pd.concat([table_1 , table_2 , table_3] , ignore_index=True)

    return data_final

In [61]:
def total_value_1(p):
    
    p.drop(p[p.description.str.contains("شرح")].index , inplace=True)
    p.drop(p[p.description.str.contains("/")].index , inplace=True)
    p.drop(p[p.value.str.contains("/")].index , inplace=True)

    p.insert(loc=4 , column='is_total_value' , value= '0')

    index=p[p.description.str.contains('total')].index
    index=index.to_list()

    p.loc[index,'is_total_value']=1

    if p.loc[0 , 'is_corrected']=='1':
        k='_correction_'
    else:
        k='_'
          
    directory = "D:/Python_project/FirmFSCleansing/processed/FSCleanedAfterUpdatingDateOfScrape_14011026"
    file_name = list(nemad.loc[nemad['en'] == p.loc[0 , 'nemad']]['en'])[0]
    path_join = os.path.join(directory, file_name)
    try:
        os.mkdir(path_join)
    except :
        print('Cant')    
        

    p.to_excel(f"{path_join}/{p.iloc[0,5]}_{p.iloc[0,6]}_{p.iloc[0,9]}_{p.iloc[0,11]}{k}balancesheet.xlsx" , index=False)

In [62]:
def final_1(path):
    total_value_1(x_concat_1(char_edit_1(time_edit_1(col_edit_1(label_trans_1(new_col_1(clean_drop_1(pd.read_excel(path)))))))))

In [63]:
# errorlist=[]
# def test_2(kk):
#     l = pd.read_excel(kk)
#     a = l.shape
#     if a == (54, 13):
#         try:
#             pd.Series(kk).apply(final_1)
#         except:
#             errorlist.append(kk)

In [64]:
# pd.Series(balance_sheet).apply(test_2)

In [65]:
# errorlist

# type_2_function

In [66]:
# 1st function
def drop_filter_concat(q):

    # drop all NaN rows and columns
    q.drop(q.columns[0], axis=1, inplace= True)
    q.dropna(axis=1 , how="all" , inplace=True)

    #split df to 3 tables
    q_part_1 = q.filter([q.columns[0] , q.columns[1] , q.columns[2] , q.columns[3]])
    q_part_2 = q.filter([q.columns[4] , q.columns[5] , q.columns[6] , q.columns[7]])
    q_part_3 = q.filter([q.columns[8] , q.columns[9] , q.columns[10] , q.columns[11] , q.columns[12] , q.columns[13]])

    #drop NaN rows
    q_part_1.dropna(axis=0 , how="all" , inplace=True)
    q_part_2.dropna(axis=0 , how="all" , inplace=True)

    #drop percent_change column it was useless
    q_part_1.drop(q_part_1.columns[3] , axis=1 , inplace=True)
    q_part_2.drop(q_part_2.columns[3] , axis=1 , inplace=True)

    #reset index
    q_part_1.reset_index(drop=True , inplace=True)
    q_part_2.reset_index(drop=True , inplace=True)
    q_part_3.reset_index(drop=True , inplace=True)

    #set same col names
    q_part_1.rename(columns={q_part_1.columns[0] : 'description' , 
                           q_part_1.columns[1] : 'value_1' ,
                           q_part_1.columns[2] : 'value_2'} , inplace=True)

    q_part_2.rename(columns={q_part_2.columns[0] : 'description' , 
                           q_part_2.columns[1] : 'value_1' ,
                           q_part_2.columns[2] : 'value_2'} , inplace=True)

    #concat
    q_concat = pd.concat([pd.concat([q_part_1 , q_part_2] , axis=0 , ignore_index=True) , q_part_3] , axis=1)

    return q_concat

In [67]:
# 2nd Finction
def date_find(d):
    date_value=[]
    try:
        if re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'value_2']).group(1) != re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'title']).group(1):
            date_value.append(re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'value_2']).group(1))
        else:
            print(False)
    except:
        print(False)

    try:
        if re.search(r'(\d+/\d+/\d+)', d.loc[1 , 'value_2']).group(1) != re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'title']).group(1):
            date_value.append(re.search(r'(\d+/\d+/\d+)', d.loc[1 , 'value_2']).group(1))
        else:
            print(False)
    except:
        print(False)

    try:
        if re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'value_1']).group(1) != re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'title']).group(1):
            date_value.append(re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'value_1']).group(1))
        else:
            print(False)
    except:
        print(False)
    
    try:
        if re.search(r'(\d+/\d+/\d+)', d.loc[1 , 'value_1']).group(1) != re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'title']).group(1):
            date_value.append(re.search(r'(\d+/\d+/\d+)', d.loc[1 , 'value_1']).group(1))
        else:
            print(False)
    except:
        print(False)




    d.insert(loc=2 , column='reported_date' , value= re.sub("\/" , "-" , digits.fa_to_en(re.search(r'(\d+/\d+/\d+)', d.loc[0 , 'title']).group(1))))
    d.insert(loc=4 , column='last_year_date' , value= re.sub("\/" , "-" ,digits.fa_to_en(date_value[0])))

    return d

In [68]:
# 3rd function
def final_concat(f):

    #audit column
    if f.loc[0,'title'].find('حسابرسی شده') != -1:   
        f.insert(loc=3 , column='reported_audit_status' , value= "audited")
    elif f.loc[0,'title'].find('حسابرسی نشده') != -1:
        f.insert(loc=3 , column='reported_audit_status' , value= "not_audited")
    else:
        f.insert(loc=3 , column='reported_audit_status' , value= "unknown")

    f.insert(loc=6 , column='reported_audit_status_2' , value= "unknown")



    #second concat for database form 
    f_table_1 = f.filter([f.columns[0] , f.columns[1] , f.columns[2] , f.columns[3] , f.columns[7] ,
                                             f.columns[8] , f.columns[9] , f.columns[10] , f.columns[11] , f.columns[12]])
    f_table_2 = f.filter([f.columns[0] , f.columns[4] , f.columns[5] , f.columns[6] , f.columns[7] ,
                                              f.columns[8] , f.columns[9] , f.columns[10] , f.columns[11] , f.columns[12]])

    f_table_1.rename(columns={f_table_1.columns[1]:'value' , f_table_1.columns[2]:'date' , f_table_1.columns[3]:'audit_status'} , inplace=True)
    f_table_2.rename(columns={f_table_2.columns[1]:'value' , f_table_2.columns[2]:'date' , f_table_2.columns[3]:'audit_status'} , inplace=True)

    f_final_concat = pd.concat([f_table_1 , f_table_2])
    
    f_final_concat.replace(np.nan , '0' , inplace=True)

    return f_final_concat

In [69]:
# 4th function
def char_correction(ch):
#     ch.replace(np.nan , '0' , inplace=True)

    ch.insert(loc= 4, column= "national_code", value= str(list(nemad.loc[nemad['stock_name']== list(ch.loc[0 , 'nemad'])[0]]['shenaseh_meli'])[0]))
    ch['company_name'] = ch['company_name'].replace(ch['company_name'].values, list(nemad.loc[nemad['stock_name']== list(ch.loc[0 , 'nemad'])[0]]['company_name_English'])[0])
    ch['nemad'] = ch['nemad'].replace(ch['nemad'].values, list(nemad.loc[nemad['stock_name']== list(ch.loc[0 , 'nemad'])[0]]['en'])[0])
    
    
       
    
#     ch.insert(loc= 4, column= "national_code", value= str(list(nemad.loc[nemad['stock_name']== list(ch.loc[0 , 'nemad'])[0]]['shenaseh_meli'])[0]))
#     ch['nemad'] = ch['nemad'].replace(ch['nemad'].values, list(nemad.loc[nemad['stock_name']== ch.loc[0 , 'nemad']]['en'][0]))
#     ch['company_name']=ch['company_name'].replace(ch['company_name'].values, list(nemad.loc[nemad['stock_name']== ch.loc[0 , 'nemad']]['company_name_English'])[0])
#     ch['company_name']= ch['company_name'].replace(ch['company_name'].values, list(nemad.loc[nemad['stock_name']== list(ch.loc[0 , 'nemad'])[0]]['company_name_English'])[0])
#     ch['nemad'] = ch['nemad'].replace(ch['nemad'].values, list(nemad.loc[nemad['stock_name']== 'آ س پ' ]['en'])[0])
#     ch['company_name']=ch['company_name'].replace(ch['company_name'].values, list(nemad.loc[nemad['stock_name']== 'آ س پ' ]['company_name_English'])[0])
    
    
    # adding national code col

    # char correction
    ch['table_title']=ch['table_title'].replace(ch['table_title'].values, "balance_sheet" )
    ch['send_time']= ch['send_time'].replace(list(ch['send_time']) , ch.iloc[0 , 8])
    ch['publish_time']= ch['publish_time'].replace(list(ch['publish_time']) , ch.iloc[0 , 9])

    #persian number to english number
    ch[ch.columns[1]]= ch[ch.columns[1]].apply(lambda x: digits.fa_to_en(x))
    ch['publish_time']= ch['publish_time'].apply(lambda x: digits.fa_to_en(x))
    ch['send_time']= ch['send_time'].apply(lambda x: digits.fa_to_en(x))

    return ch

In [70]:
# 6th function
def title_split(j):
  
    # get report period from title
    if j.iloc[0,7].find('دوره ۶ ماهه') != -1:
        j.insert(loc=7 , column = 'reported_period' , value = "6_months_interim")
    elif j.iloc[0,7].find('دوره ۳ ماهه') != -1:
        j.insert(loc=7 , column='reported_period' , value= "3_months_interim")
    elif j.iloc[0,7].find('دوره ۹ ماهه') != -1:
        j.insert(loc=7 , column='reported_period' , value= "9_months_interim")
    else:
        j.insert(loc=7 , column='reported_period' , value= "12_months_interim")


    # get report date from title
    j.insert(loc=8 , column="reported_date_j" , value =re.sub("\/" , "-" , digits.fa_to_en(re.search(r'(\d+/\d+/\d+)', j.iloc[0 , 8]).group(1))) )
    j.insert(loc=9 , column="reported_date_m" , value= j['reported_date_j'].apply(lambda x:to_g(x)))


    # get audit status from title
    if j.iloc[0 , 10].find('حسابرسی شده') != -1:
        j.insert(loc=10 , column='reported_audit_status' , value= "audited")
    elif j.iloc[0, 10].find('حسابرسی نشده') != -1:
        j.insert(loc=10 , column='reported_audit_status' , value= "not_audited")
    else:
        j.insert(loc=10 , column='reported_audit_status' , value= "unknown")

    if j.iloc[0 , 11].find('اصلاحیه') != -1:
        j.insert(loc=11 , column= 'is_corrected' , value='1')
    else:
        j.insert(loc=11 , column= 'is_corrected' , value='0')



    j.drop(columns=['title'] , inplace=True)

    return j

In [71]:
# 8th function
def final_drop(ff):

    ff.drop(ff[ff.description.str.contains("شرح")].index , inplace=True)
#     ff.drop(ff[ff.description.str.contains("۱۴" or  "۱۳" or "14" or "13")].index , inplace=True)
#     ff.drop(ff[ff.value.str.contains("۱۴" or  "۱۳" or "14" or "13")].index , inplace=True)

    ff.drop(ff[ff.description.str.contains("/")].index , inplace=True)
    ff.drop(ff[ff.value.str.contains("/")].index , inplace=True)
    
    

    ff.reset_index(drop=True , inplace=True)

    return ff

In [72]:
# 9th function
def total_value(pp):

    pp.insert(loc=4 , column='is_total_value' , value= '0')

    index=pp[pp.description.str.contains('total')].index
    index=index.to_list()

    pp.loc[index,'is_total_value']=1

    if pp.loc[0 , 'is_corrected']=='1':
        k='_correction_'
    else:
        k='_'


    directory = "D:/Python_project/FirmFSCleansing/processed/FSCleanedAfterUpdatingDateOfScrape_14011026"
    file_name = list(nemad.loc[nemad['en'] == pp.loc[0 , 'nemad']]['en'])[0]
    path_join = os.path.join(directory, file_name)
    try:
        os.mkdir(path_join)
    except :
        print('Cant')


        
    pp.to_excel(f"{path_join}/{pp.iloc[0,5]}_{pp.iloc[0,6]}_{pp.iloc[0,9]}_{pp.iloc[0,11]}{k}balancesheet.xlsx" , index=False)

In [73]:
def export(path):
    total_value(final_drop(label_trans_1(title_split(char_correction(final_concat(date_find(drop_filter_concat(pd.read_excel(path)))))))))

# get_export_by_shape

In [74]:
errorlist_1 = []
errorlist_2 = []
errorlist_3 = []
errorlist_4 = []
errorlist_5 = []
errorlist_6 = []
errorlist_7 = []
errorlist_7 = []
errorlist_8 = []
errorlist_9 = []
errorlist_10 = []
errorlist_11 = []
errorlist_12 = []
errorlist_13 = []




def test_1(kk):
    l = pd.read_excel(kk)
    a = l.shape
    
    if a == (38, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_1.append(kk)

    if a == (44, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_2.append(kk)
            
    if a == (36, 15):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_3.append(kk)
            
    if a == (37, 15):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_4.append(kk)
            
    if a == (34, 15):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_5.append(kk)
            
    if a == (47, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_6.append(kk)
                        
    if a == (43, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_7.append(kk)
            
    if a == (41, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_8.append(kk)
            
                            
    if a == (39, 17):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_9.append(kk)
            
                
    if a == (36, 15):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_10.append(kk)
                   
    if a == (32, 15):
        try:
            pd.Series(kk).apply(export)
        except:
            errorlist_11.append(kk) 
                   
    if a == (43, 13):
        try:
            pd.Series(kk).apply(final_1)
        except:
            errorlist_12.append(kk)
                
    if a == (54, 13):
        try:
            pd.Series(kk).apply(final_1)
        except:
            errorlist_13.append(kk)

In [75]:
pd.Series(balance_sheet).apply(test_1)

Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
Cant
Cant
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
Cant
Cant
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
False
False
False
Cant
False
False
False
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
False
False
False
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
Cant
Cant
Cant
Cant
Cant
False
False
False
Cant
False
False
False
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
Cant
Cant
Cant
Cant
Cant
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
Cant
False
False
False
False
False
False
False
Cant
False
False
False
Cant
F

0       None
1       None
2       None
3       None
4       None
        ... 
1263    None
1264    None
1265    None
1266    None
1267    None
Length: 1268, dtype: object

In [76]:
for i in [errorlist_1,errorlist_2,errorlist_3,errorlist_4,errorlist_5,errorlist_6,errorlist_7,errorlist_8,errorlist_9,errorlist_10,errorlist_11,errorlist_12,errorlist_13]:
    print(len(i))

0
0
0
0
9
0
0
0
0
0
0
0
13


In [ ]:
for i in [errorlist_1,errorlist_2,errorlist_3,errorlist_4,errorlist_5,errorlist_6,errorlist_7,errorlist_8,errorlist_9,errorlist_10,errorlist_11,errorlist_12,errorlist_13]:
    print(len(i))

0
0
0
0
9
0
0
0
0
0
0
0
13
